In [ ]:
!pip install geopandas

In [ ]:
!pip install geopy

Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for geopy from https://files.pythonhosted.org/packages/e5/15/cf2a69ade4b194aa524ac75112d5caac37414b20a3a03e6865dfe0bd1539/geopy-2.4.1-py3-none-any.whl.metadata
  Obtaining dependency information for geographiclib<3,>=1.52 from https://files.pythonhosted.org/packages/9f/5a/a26132406f1f40cf51ea349a5f11b0a46cec02a2031ff82e391c2537247a/geographiclib-2.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   ---------------------------------------- 0.0/125.4 kB ? eta -:--:--
   --- ------------------------------------ 10.2/125.4 kB ? eta -:--:--
   --------- ----------------------------- 30.7/125.4 kB 435.7 kB/s eta 0:00:01
   ------------ -------------------------- 41.0/125.4 kB 281.8 kB/s eta 0:00:01
   ------------------- ------------------- 61.4/125.4 kB 297.7 k


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [100]:
from datetime import datetime
import geopandas as gpd
import pandas as pd
import folium
import matplotlib.pyplot as plt
from folium.plugins import MarkerCluster
from shapely.geometry import Polygon, Point
import numpy as np
from scipy.spatial.distance import cdist
import math
from geopy.distance import geodesic
from dataprep.eda import create_report
from folium.plugins import HeatMap

In [ ]:
df = gpd.read_file('2023-10-01_performance_fixed_tiles.zip')


In [ ]:
networkdf = df
citiesdf = pd.read_csv('worldcities.csv')

Preprocessing Cities Data to extract the average long and lang of Egyptian Cities

In [ ]:
citiesdf.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37732000.0,1392685764
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629


Filtering data to only contain egyptian cities

In [ ]:
citiesdf.drop(['city_ascii', 'iso2', 'iso3', 'admin_name', 'capital', 'population', 'id'], axis=1, inplace=True)
citiesdf = citiesdf[citiesdf['country'] == 'Egypt']
print(len(citiesdf))
citiesdf.head(10)


159


,city,lat,lng,country
10,Cairo,30.0444,31.2358,Egypt
110,Giza,29.9870,31.2118,Egypt
145,Alexandria,31.1975,29.8925,Egypt
745,Shubrā al Khaymah,30.1286,31.2422,Egypt
935,Al Manşūrah,31.0500,31.3833,Egypt
1300,Ḩalwān,29.8453,31.3333,Egypt
1408,Al Maḩallah al Kubrá,30.9686,31.1636,Egypt
1496,Port Said,31.2625,32.3061,Egypt
1509,Suez,29.9667,32.5500,Egypt
1528,Al ‘Āshir min Ramaḑān,30.3065,31.7415,Egypt


In [ ]:
citiesdf.drop('country', axis=1, inplace=True)

plotting the cities on the egyptian map

In [ ]:
citiesgeodf = gpd.GeoDataFrame(citiesdf, geometry=gpd.points_from_xy(citiesdf.lng, citiesdf.lat))

egypt_map = folium.Map(location=[26.8206, 30.802498], zoom_start=6)

# Create a MarkerCluster layer for the city points
marker_cluster = MarkerCluster().add_to(egypt_map)

# Add each city point to the MarkerCluster layer
for idx, row in citiesdf.iterrows():
    folium.Marker(
        location=[row['lat'], row['lng']],
        popup=row['city'],  # You can customize the popup content
    ).add_to(marker_cluster)

# Display the map
egypt_map

preprocessing the network speed df

In [ ]:
print(networkdf.head())

   avg_d_kbps  avg_u_kbps  avg_lat_ms  tests  devices  \
0      139973        7796          92      1        1   
1      105320       14528         123     10        1   
2      155124       14996         107      1        1   
3      135038       14319         108      1        1   
4      165698       10831          83      1        1   

                                            geometry  
0  POLYGON ((-160.02686 70.64359, -160.02136 70.6...  
1  POLYGON ((-160.03784 70.63995, -160.03235 70.6...  
2  POLYGON ((-160.03235 70.63995, -160.02686 70.6...  
3  POLYGON ((-160.04333 70.63631, -160.03784 70.6...  
4  POLYGON ((-160.03784 70.63813, -160.03235 70.6...  


In [ ]:
networkdf["centrod"] = networkdf['geometry'].centroid

In [ ]:
print(networkdf.head())

   avg_d_kbps  avg_u_kbps  avg_lat_ms  tests  devices  \
0      139973        7796          92      1        1   
1      105320       14528         123     10        1   
2      155124       14996         107      1        1   
3      135038       14319         108      1        1   
4      165698       10831          83      1        1   

                                            geometry  \
0  POLYGON ((-160.02686 70.64359, -160.02136 70.6...   
1  POLYGON ((-160.03784 70.63995, -160.03235 70.6...   
2  POLYGON ((-160.03235 70.63995, -160.02686 70.6...   
3  POLYGON ((-160.04333 70.63631, -160.03784 70.6...   
4  POLYGON ((-160.03784 70.63813, -160.03235 70.6...   

                       centrod  
0  POINT (-160.02411 70.64268)  
1  POINT (-160.03510 70.63904)  
2  POINT (-160.02960 70.63904)  
3  POINT (-160.04059 70.63539)  
4  POINT (-160.03510 70.63722)  


In [ ]:
bbox_polygon = Polygon([(25.00701162873365, 28.06619072339427),
                        (34.77285589154759, 28.8285422405373),
                        (30.569922352416782, 31.683835883824027),
                        (30.94490956324461, 22.018074272617476)])

bbox_df = gpd.GeoDataFrame(geometry=[bbox_polygon], crs=networkdf.crs)
bbox_df = bbox_df.to_crs(networkdf.crs)
filtered_networks_df = networkdf[networkdf.intersects(bbox_df.unary_union)]
filtered_networks_df

,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,geometry,centrod
4428701,91528,94732,4,1,1,"POLYGON ((27.56470 27.57159, 27.57019 27.57159...",POINT (27.56744 27.56916)
4428702,76459,64054,17,1,1,"POLYGON ((27.92175 27.13737, 27.92725 27.13737...",POINT (27.92450 27.13492)
4428703,24891,15299,13,7,3,"POLYGON ((27.87231 27.06402, 27.87781 27.06402...",POINT (27.87506 27.06157)
4428704,36193,56719,14,4,2,"POLYGON ((27.88879 27.06891, 27.89429 27.06891...",POINT (27.89154 27.06646)
4428705,101906,80136,11,10,2,"POLYGON ((27.88879 27.06402, 27.89429 27.06402...",POINT (27.89154 27.06157)
...,...,...,...,...,...,...,...
4454264,14159,1527,22,1,1,"POLYGON ((30.53650 25.20494, 30.54199 25.20494...",POINT (30.53925 25.20246)
4454265,26306,7921,14,4,1,"POLYGON ((30.58594 25.22482, 30.59143 25.22482...",POINT (30.58868 25.22234)
4454266,10221,517,28,4,1,"POLYGON ((30.59692 24.68695, 30.60242 24.68695...",POINT (30.59967 24.68446)
4454267,33864,4463,59,2,2,"POLYGON ((30.59692 24.67697, 30.60242 24.67697...",POINT (30.59967 24.67447)


In [ ]:
len(filtered_networks_df)

17871

In [ ]:
filtered_networks_df.drop('geometry', axis  = 1 , inplace= True)

In [ ]:
filtered_networks_df.head()

,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,centrod
4428701,91528,94732,4,1,1,POINT (27.56744 27.56916)
4428702,76459,64054,17,1,1,POINT (27.92450 27.13492)
4428703,24891,15299,13,7,3,POINT (27.87506 27.06157)
4428704,36193,56719,14,4,2,POINT (27.89154 27.06646)
4428705,101906,80136,11,10,2,POINT (27.89154 27.06157)


In [ ]:
filtered_networks_df['centroid_lat'] = filtered_networks_df['centrod'].y
filtered_networks_df['centroid_lon'] = filtered_networks_df['centrod'].x

In [ ]:
filtered_networks_df.drop('centrod', axis = 1 , inplace= True)

In [ ]:
filtered_networks_df[filtered_networks_df.duplicated()]

,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,centroid_lat,centroid_lon


In [ ]:
filtered_networks_df.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Int64Index: 17871 entries, 4428701 to 4454268
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   avg_d_kbps    17871 non-null  int64  
 1   avg_u_kbps    17871 non-null  int64  
 2   avg_lat_ms    17871 non-null  int64  
 3   tests         17871 non-null  int64  
 4   devices       17871 non-null  int64  
 5   centroid_lat  17871 non-null  float64
 6   centroid_lon  17871 non-null  float64
dtypes: float64(2), int64(5)
memory usage: 1.1 MB


In [ ]:
citiesdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 159 entries, 10 to 44657
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   city    159 non-null    object 
 1   lat     159 non-null    float64
 2   lng     159 non-null    float64
dtypes: float64(2), object(1)
memory usage: 5.0+ KB


Calculating the distance between the filtered data and the different cities in the cities df

In [ ]:
def haversine(lat1, lon1, lat2, lon2):

    dlat = (lat2-lat1) * math.pi / 180.0
    dlon = (lon2-lon1) * math.pi / 180
    
    lat1 = (lat1) * math.pi / 180.0
    lat2 = (lat2) * math.pi / 180.0

    a = (pow(math.sin(dlat / 2), 2) + pow(math.sin(dlon / 2),2) * math.cos(lat1) * math.cos(lat2) )
    
    radius = 6371 #radius of earth
    c = 2 * math.asin(math.sqrt(a))

    return radius * c

In [83]:
filtered_networks_df.reset_index(inplace = True)
citiesdf.reset_index(inplace = True)

In [ ]:
for i in range(len(filtered_networks_df)):
    row = filtered_networks_df.iloc[i]
    nearness_list = []
    lon = row["centroid_lon"]
    lat = row["centroid_lat"] 
    for j in range(len(citiesdf)):
        lon_city = citiesdf.iloc[j]["lng"]
        lat_city = citiesdf.iloc[j]["lat"]
        distance = haversine(lon_city, lat_city, lon, lat)
        nearness_list.append((citiesdf.iloc[j]['city'],distance))
    nearness_list.sort(key = lambda x: x[1])
    nearest_city = nearness_list[0][0]
    filtered_networks_df.at[i, 'city'] = nearest_city

In [ ]:
filtered_networks_df['city'] = filtered_networks_df.apply(
    lambda row: citiesdf.iloc[
        np.argmin([geodesic((row["centroid_lat"], row["centroid_lon"]), (lat, lon)).km for lat, lon in zip(citiesdf['lat'], citiesdf['lng'])])
    ]['city'], axis=1
)

In [101]:
#save filtered-networks.csv file
filtered_networks_df.to_csv('filtered-networks.csv')

In [100]:
filtered_networks_df['city'].value_counts()

Al Khānkah                      1509
Al ‘Āshir min Ramaḑān           1025
Madīnat as Sādis min Uktūbar     724
Kirdāsah                         576
Ḩalwān                           532
                                ... 
Al Qūşīyah                         7
Sawādah                            5
Rosetta                            2
Al Qays                            1
Maţāy                              1
Name: city, Length: 89, dtype: int64

In [23]:
filtered_networks_df.head()

,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,centroid_lat,centroid_lon,city,avg_d_speed_mbps,avg_u_speed_mbps
0,91528,94732,4,1,1,27.569156,27.567444,Qaşr al Farāfirah,91.528,94.732
1,76459,64054,17,1,1,27.134924,27.924500,Qaşr al Farāfirah,76.459,64.054
2,24891,15299,13,7,3,27.061572,27.875061,Qaşr al Farāfirah,24.891,15.299
3,36193,56719,14,4,2,27.066463,27.891541,Qaşr al Farāfirah,36.193,56.719
4,101906,80136,11,10,2,27.061572,27.891541,Qaşr al Farāfirah,101.906,80.136


In [13]:
filtered_networks_df.drop(['Unnamed: 0', 'index'], axis = 1, inplace= True)

In [14]:
filtered_networks_df['avg_d_speed_mbps']  = filtered_networks_df['avg_d_kbps']/1000
filtered_networks_df['avg_u_speed_mbps'] = filtered_networks_df['avg_u_kbps']/1000

In [15]:
filtered_networks_df.describe()

,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,centroid_lat,centroid_lon,avg_d_speed_mbps,avg_u_speed_mbps
count,17871.000000,17871.000000,17871.000000,17871.000000,17871.000000,17871.000000,17871.000000,17871.000000,17871.000000
mean,24320.552739,9353.365285,37.065637,42.168653,11.246936,30.253199,31.245289,24.320553,9.353365
std,39175.021429,22929.843534,72.937887,140.815593,102.817435,0.685185,0.409037,39.175021,22.929844
min,4.000000,4.000000,0.000000,1.000000,1.000000,24.674474,27.567444,0.004000,0.004000
25%,9288.000000,1275.000000,13.000000,2.000000,1.000000,30.004895,30.989685,9.288000,1.275000
50%,16839.000000,2928.000000,21.000000,7.000000,2.000000,30.299389,31.231384,16.839000,2.928000
75%,24440.000000,5192.500000,35.000000,31.000000,8.000000,30.696973,31.451111,24.440000,5.192500
max,791794.000000,491476.000000,2456.000000,13422.000000,13311.000000,31.524703,33.192444,791.794000,491.476000


In [38]:
filtered_networks_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17871 entries, 0 to 17870
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   avg_d_kbps        17871 non-null  int64  
 1   avg_u_kbps        17871 non-null  int64  
 2   avg_lat_ms        17871 non-null  int64  
 3   tests             17871 non-null  int64  
 4   devices           17871 non-null  int64  
 5   centroid_lat      17871 non-null  float64
 6   centroid_lon      17871 non-null  float64
 7   city              17871 non-null  object 
 8   avg_d_speed_mbps  17871 non-null  float64
 9   avg_u_speed_mbps  17871 non-null  float64
dtypes: float64(4), int64(5), object(1)
memory usage: 1.4+ MB


In [41]:
filtered_networks_df.value_counts()

avg_d_kbps  avg_u_kbps  avg_lat_ms  tests  devices  centroid_lat  centroid_lon  city                   avg_d_speed_mbps  avg_u_speed_mbps
4           700         15          1      1        30.914007     30.753479     Şā al Ḩajar            0.004             0.700               1
21539       3830        19          49     15       30.171250     31.346741     Al Khānkah             21.539            3.830               1
21545       11580       55          33     11       31.083518     30.951233     Sakhā                  21.545            11.580              1
            2721        23          9      3        30.654453     30.984192     Shibīn al Kawm         21.545            2.721               1
21542       4188        15          148    52       30.000138     31.423645     Ḩalwān                 21.542            4.188               1
                                                                                                                                            ..
1188

In [ ]:
df['city']

In [49]:
groupedby_city = filtered_networks_df.groupby('city').count()
groupedby_city.sort_values(by='tests', ascending=False).head(15)

,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,centroid_lat,centroid_lon,avg_d_speed_mbps,avg_u_speed_mbps
city,,,,,,,,,
Al Khānkah,1509,1509,1509,1509,1509,1509,1509,1509,1509
Al ‘Āshir min Ramaḑān,1025,1025,1025,1025,1025,1025,1025,1025,1025
Madīnat as Sādis min Uktūbar,724,724,724,724,724,724,724,724,724
Kirdāsah,576,576,576,576,576,576,576,576,576
Ḩalwān,532,532,532,532,532,532,532,532,532
Bilbays,526,526,526,526,526,526,526,526,526
Cairo,478,478,478,478,478,478,478,478,478
Suez,418,418,418,418,418,418,418,418,418
Ţanţā,406,406,406,406,406,406,406,406,406


In [53]:
filtered_networks_df['city'] = filtered_networks_df['city'].replace({'Ḩalwān': 'Cairo', 'Kirdāsah': 'Cairo'})

In [55]:
groupedby_city = filtered_networks_df.groupby('city').count()
groupedby_city.sort_values(by='tests', ascending=False).head(5)

,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,centroid_lat,centroid_lon,avg_d_speed_mbps,avg_u_speed_mbps
city,,,,,,,,,
Cairo,1586,1586,1586,1586,1586,1586,1586,1586,1586
Al Khānkah,1509,1509,1509,1509,1509,1509,1509,1509,1509
Al ‘Āshir min Ramaḑān,1025,1025,1025,1025,1025,1025,1025,1025,1025
Madīnat as Sādis min Uktūbar,724,724,724,724,724,724,724,724,724
Bilbays,526,526,526,526,526,526,526,526,526


In [54]:
filtered_networks_df.to_csv('filtered-networks.csv')

In [56]:
filtered_networks_df

,avg_d_kbps,avg_u_kbps,avg_lat_ms,tests,devices,centroid_lat,centroid_lon,city,avg_d_speed_mbps,avg_u_speed_mbps
0,91528,94732,4,1,1,27.569156,27.567444,Qaşr al Farāfirah,91.528,94.732
1,76459,64054,17,1,1,27.134924,27.924500,Qaşr al Farāfirah,76.459,64.054
2,24891,15299,13,7,3,27.061572,27.875061,Qaşr al Farāfirah,24.891,15.299
3,36193,56719,14,4,2,27.066463,27.891541,Qaşr al Farāfirah,36.193,56.719
4,101906,80136,11,10,2,27.061572,27.891541,Qaşr al Farāfirah,101.906,80.136
...,...,...,...,...,...,...,...,...,...,...
17866,14159,1527,22,1,1,25.202456,30.539246,Al Khārjah,14.159,1.527
17867,26306,7921,14,4,1,25.222335,30.588684,Al Khārjah,26.306,7.921
17868,10221,517,28,4,1,24.684457,30.599670,Al Khārjah,10.221,0.517
17869,33864,4463,59,2,2,24.674474,30.599670,Al Khārjah,33.864,4.463


In [101]:
import folium

gdf = gpd.GeoDataFrame(filtered_networks_df, geometry=gpd.points_from_xy(filtered_networks_df['centroid_lon'], filtered_networks_df['centroid_lat']))
mymap = folium.Map(location=[gdf['centroid_lat'].mean(), gdf['centroid_lon'].mean()], zoom_start=2)
heat_data = [[point.xy[1][0], point.xy[0][0], speed] for point, speed in zip(gdf.geometry, gdf['avg_d_speed_mbps'])]
HeatMap(heat_data, name='Heatmap', radius=15).add_to(mymap)

mymap.save('interactive_heatmap.html')  # Save the map as an HTML file
mymap